Sunday, Feburary 26, 2023

https://huggingface.co/course/chapter8/2?fw=pt

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

In [3]:
pushToHub = False

In [4]:
# only target the 2070 Super ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# What to do when you get an error

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [5]:
# from huggingface_hub import notebook_login

# notebook_login()

In [6]:
# !pip install datasets evaluate transformers[sentencepiece]
# !apt install git-lfs

You will need to setup git, adapt your email and name in the following cell.

In [7]:
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [8]:
# from huggingface_hub import notebook_login

# notebook_login()

In [9]:
from distutils.dir_util import copy_tree
from huggingface_hub import Repository, snapshot_download, create_repo, get_full_repo_name


def copy_repository_template():
    # Clone the repo and extract the local path
    template_repo_id = "lewtun/distilbert-base-uncased-finetuned-squad-d5716d28"
    commit_hash = "be3eaffc28669d7932492681cd5f3e8905e358b4"
    template_repo_dir = snapshot_download(template_repo_id, revision=commit_hash)
    # Create an empty repo on the Hub
    model_name = template_repo_id.split("/")[1]
    create_repo(model_name, exist_ok=True)
    # Clone the empty repo
    new_repo_id = get_full_repo_name(model_name)
    new_repo_dir = model_name
    repo = Repository(local_dir=new_repo_dir, clone_from=new_repo_id)
    # Copy files
    copy_tree(template_repo_dir, new_repo_dir)
    # Push to Hub
    repo.push_to_hub()

In [10]:
from transformers import pipeline

model_checkpoint = get_full_repo_name("distillbert-base-uncased-finetuned-squad-d5716d28")
# reader = pipeline("question-answering", model=model_checkpoint)

In [11]:
reader = pipeline("question-answering", model=model_checkpoint)

OSError: robkayinto/distillbert-base-uncased-finetuned-squad-d5716d28 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
model_checkpoint = get_full_repo_name("distilbert-base-uncased-finetuned-squad-d5716d28")
#reader = pipeline("question-answering", model=model_checkpoint)

In [ ]:
reader = pipeline("question-answering", model=model_checkpoint)

In [ ]:
from huggingface_hub import list_repo_files

list_repo_files(repo_id=model_checkpoint)

In [ ]:
from transformers import AutoConfig

pretrained_checkpoint = "distilbert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_checkpoint)

In [ ]:
if pushToHub:
    config.push_to_hub(model_checkpoint, commit_message="Add config.json")

In [ ]:
reader = pipeline("question-answering", model=model_checkpoint, revision="main")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a question answering dataset is the SQuAD
dataset, which is entirely based on that task. If you would like to fine-tune a
model on a SQuAD task, you may leverage the
examples/pytorch/question-answering/run_squad.py script.

🤗 Transformers is interoperable with the PyTorch, TensorFlow, and JAX
frameworks, so you can use your favourite tools for a wide variety of tasks!
"""

question = "What is extractive question answering?"
reader(question=question, context=context)

In [ ]:
tokenizer = reader.tokenizer
model = reader.model

In [ ]:
question = "Which frameworks can I use?"

In [ ]:
import torch

inputs = tokenizer(question, context, add_special_tokens=True)
input_ids = inputs["input_ids"][0]
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits
# Get the most likely beginning of answer with the argmax of the score
answer_start = torch.argmax(answer_start_scores)
# Get the most likely end of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
)
print(f"Question: {question}")
print(f"Answer: {answer}")

In [ ]:
inputs["input_ids"][:5]

In [ ]:
type(inputs["input_ids"])

In [ ]:
endTime = time.time()

elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))

print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")